In [1]:
import numpy as np
import torch
import copy
from metabci.brainda.algorithms.deep_learning import EEGNet_Sparse
from metabci.brainda.algorithms.utils.model_selection import (
    set_random_seeds,
    generate_kfold_indices, match_kfold_indices)
from metabci.brainda.datasets import Wang2016, BETA
# 三个SSVEP Datasets：Nakanishi2015、Wang2016、BETA
from metabci.brainda.paradigms import SSVEP
from skorch.helper import predefined_split

In [2]:
#**************************************************
# Benchmark数据集读取处理
#**************************************************
Bench_dataset = Wang2016()
subject_list = list(range(1, 21))  # 被试编号从1到20
for s in subject_list:
    Bench_dataset.data_path(subject=s, path="E:\\MetaBCI-master\\mne_data")  # 依次为每个被试设置路径
events = Bench_dataset.events.keys()
freq_list = [str(Bench_dataset.get_freq(event)) for event in events]  # 获得所有刺激的频率
freq_map = {i: freq for i, freq in enumerate(freq_list)}  # 标签到频率的映射

Bench_subjects = list(range(1, 21))  # 使用S1–S20被试进行训练
Bench_paradigm = SSVEP(
    channels=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],  # 选择电极通道
    intervals=[(0.14, 1.14)],  # 分析时间窗最长-0.5-5.5s ,0.0是刺激开始
    events=freq_list,  # 选择所有刺激频率
    srate=250  # 采样率
)

# add 5-90Hz bandpass filter in raw hook
# 对原始EEG信号做5-90Hz带通滤波。
def raw_hook(raw, caches):
    raw.filter(5, 90, l_trans_bandwidth=2, h_trans_bandwidth=5, phase='zero-double')
    caches['raw_stage'] = caches.get('raw_stage', -1) + 1
    return raw, caches

Bench_paradigm.register_raw_hook(raw_hook)



# 获取微调数据
X_Bench, y_Bench, meta_Bench = Bench_paradigm.get_data(
    Bench_dataset,
    subjects=Bench_subjects,
    return_concat=True,
    n_jobs=None,
    verbose=False
)

--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun/S5.mat.7z
--------ssssss, /upload/yijun/S6.mat.7z
--------ssssss, /upload/yijun/S7.mat.7z
--------ssssss, /upload/yijun/S8.mat.7z
--------ssssss, /upload/yijun/S9.mat.7z
--------ssssss, /upload/yijun/S10.mat.7z
--------ssssss, /upload/yijun/S11.mat.7z
--------ssssss, /upload/yijun/S12.mat.7z
--------ssssss, /upload/yijun/S13.mat.7z
--------ssssss, /upload/yijun/S14.mat.7z
--------ssssss, /upload/yijun/S15.mat.7z
--------ssssss, /upload/yijun/S16.mat.7z
--------ssssss, /upload/yijun/S17.mat.7z
--------ssssss, /upload/yijun/S18.mat.7z
--------ssssss, /upload/yijun/S19.mat.7z
--------ssssss, /upload/yijun/S20.mat.7z
--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun

In [3]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 10

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_10_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0240        4.0879       0.0375        3.6894     +  0.0010  1.2948
      2       0.0288        3.8808       0.0312        3.6804     +  0.0010  0.7459
      3       0.0257        3.8303       0.0338        3.6733     +  0.0010  0.7574
      4       0.0297        3.7862       0.0450        3.6662     +  0.0010  0.7776
      5       0.0288        3.7625       0.0550        3.6588     +  0.0010  0.7456
      6       0.0333        3.7315       0.0650        3.6509     +  0.0010  0.7199
      7       0.0340        3.7235       0.0688        3.6425     +  0.0010  0.7334
      8       0.0357        3.7085       0.0875        3.6314     +  0.0010  0.7458
      9       0.0390        3.6949       0.1000        3.6194     +  0.0010  0.7457


In [4]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 20

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_20_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0257        3.9007       0.0325        3.6869     +  0.0010  0.7392
      2       0.0328        3.7703       0.0350        3.6742     +  0.0010  0.7377
      3       0.0315        3.7349       0.0525        3.6604     +  0.0010  0.7373
      4       0.0345        3.7132       0.0700        3.6451     +  0.0010  0.7439
      5       0.0450        3.6872       0.0838        3.6298     +  0.0010  0.7440
      6       0.0510        3.6675       0.0975        3.6099     +  0.0010  0.7435
      7       0.0540        3.6374       0.1200        3.5864     +  0.0010  0.7341
      8       0.0622        3.6114       0.1375        3.5557     +  0.0010  0.7402
      9       0.0818        3.5776       0.1487        3.5172     +  0.0010  0.7417


In [5]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 30

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_30_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0272        3.8337       0.0312        3.6860     +  0.0010  0.7473
      2       0.0343        3.7332       0.0462        3.6691     +  0.0010  0.7383
      3       0.0382        3.6987       0.0625        3.6499     +  0.0010  0.7457
      4       0.0432        3.6672       0.0875        3.6231     +  0.0010  0.7392
      5       0.0612        3.6340       0.1050        3.5930     +  0.0010  0.7293
      6       0.0693        3.6027       0.1212        3.5514     +  0.0010  0.7275
      7       0.0872        3.5491       0.1625        3.4958     +  0.0010  0.7377
      8       0.1217        3.4826       0.1862        3.4187     +  0.0010  0.7416
      9       0.1415        3.4198       0.2013        3.3263     +  0.0010  0.7462


In [6]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 40

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_40_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0253        3.7891       0.0375        3.6844     +  0.0010  0.7290
      2       0.0323        3.7159       0.0500        3.6656     +  0.0010  0.7399
      3       0.0390        3.6819       0.0688        3.6442     +  0.0010  0.7379
      4       0.0540        3.6519       0.0963        3.6147     +  0.0010  0.7609
      5       0.0673        3.6197       0.1175        3.5787     +  0.0010  0.7542
      6       0.0772        3.5852       0.1475        3.5294     +  0.0010  0.7275
      7       0.1103        3.5316       0.1812        3.4591     +  0.0010  0.7366
      8       0.1368        3.4604       0.1963        3.3656     +  0.0010  0.7380
      9       0.1640        3.3939       0.2200        3.2707     +  0.0010  0.7485


In [7]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 50

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_50_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0250        3.7575       0.0362        3.6849     +  0.0010  0.7582
      2       0.0315        3.7046       0.0537        3.6658     +  0.0010  0.7627
      3       0.0380        3.6782       0.0737        3.6441     +  0.0010  0.7459
      4       0.0542        3.6539       0.0925        3.6138     +  0.0010  0.7422
      5       0.0663        3.6270       0.1187        3.5767     +  0.0010  0.7459
      6       0.0770        3.5969       0.1437        3.5255     +  0.0010  0.7537
      7       0.1105        3.5499       0.1713        3.4519     +  0.0010  0.7666
      8       0.1335        3.4872       0.1900        3.3549     +  0.0010  0.7539
      9       0.1557        3.4271       0.2100        3.2587     +  0.0010  0.7374


In [8]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 60

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_60_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0250        3.7393       0.0338        3.6854     +  0.0010  0.7623
      2       0.0307        3.6984       0.0512        3.6664     +  0.0010  0.7667
      3       0.0385        3.6768       0.0725        3.6445     +  0.0010  0.7376
      4       0.0512        3.6564       0.0912        3.6139     +  0.0010  0.7414
      5       0.0620        3.6336       0.1163        3.5764     +  0.0010  0.7458
      6       0.0747        3.6069       0.1388        3.5243     +  0.0010  0.7541
      7       0.1050        3.5654       0.1625        3.4496     +  0.0010  0.7664
      8       0.1303        3.5097       0.1812        3.3514     +  0.0010  0.7417
      9       0.1512        3.4548       0.1988        3.2555     +  0.0010  0.7436


In [9]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 70

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_70_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0248        3.7278       0.0375        3.6859     +  0.0010  0.7499
      2       0.0297        3.6947       0.0500        3.6669     +  0.0010  0.7499
      3       0.0390        3.6765       0.0762        3.6449     +  0.0010  0.7547
      4       0.0488        3.6591       0.0900        3.6143     +  0.0010  0.7585
      5       0.0595        3.6392       0.1125        3.5768     +  0.0010  0.7454
      6       0.0737        3.6153       0.1400        3.5244     +  0.0010  0.7413
      7       0.1000        3.5783       0.1588        3.4498     +  0.0010  0.7500
      8       0.1270        3.5283       0.1775        3.3514     +  0.0010  0.7586
      9       0.1465        3.4781       0.1900        3.2562     +  0.0010  0.7625


In [10]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 80

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_80_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0245        3.7200       0.0350        3.6863     +  0.0010  0.7424
      2       0.0297        3.6923       0.0525        3.6675     +  0.0010  0.7353
      3       0.0390        3.6765       0.0700        3.6454     +  0.0010  0.7441
      4       0.0488        3.6614       0.0875        3.6148     +  0.0010  0.7541
      5       0.0555        3.6438       0.1050        3.5772     +  0.0010  0.7611
      6       0.0732        3.6221       0.1325        3.5248     +  0.0010  0.7244
      7       0.1000        3.5886       0.1575        3.4501     +  0.0010  0.7292
      8       0.1237        3.5432       0.1775        3.3521     +  0.0010  0.7371
      9       0.1427        3.4967       0.1875        3.2576     +  0.0010  0.7376


In [11]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 90

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_90_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0245        3.7145       0.0362        3.6866     +  0.0010  0.7540
      2       0.0305        3.6907       0.0512        3.6681     +  0.0010  0.7503
      3       0.0385        3.6767       0.0700        3.6459     +  0.0010  0.7499
      4       0.0493        3.6633       0.0850        3.6154     +  0.0010  0.7465
      5       0.0558        3.6476       0.1050        3.5779     +  0.0010  0.7365
      6       0.0705        3.6277       0.1288        3.5254     +  0.0010  0.7308
      7       0.0993        3.5972       0.1550        3.4509     +  0.0010  0.7419
      8       0.1202        3.5556       0.1737        3.3536     +  0.0010  0.7485
      9       0.1390        3.5123       0.1888        3.2601     +  0.0010  0.7537


In [12]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 100

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_100_0.2.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0240        3.7103       0.0338        3.6870     +  0.0010  0.8545
      2       0.0310        3.6895       0.0500        3.6686     +  0.0010  0.8210
      3       0.0382        3.6770       0.0700        3.6465     +  0.0010  0.8249
      4       0.0488        3.6651       0.0825        3.6160     +  0.0010  0.7914
      5       0.0555        3.6508       0.1025        3.5785     +  0.0010  0.7566
      6       0.0685        3.6325       0.1288        3.5261     +  0.0010  0.8035
      7       0.0995        3.6044       0.1525        3.4519     +  0.0010  0.7957
      8       0.1182        3.5660       0.1713        3.3554     +  0.0010  0.8253
      9       0.1368        3.5256       0.1875        3.2627     +  0.0010  0.7517


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_10_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 10
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 63.7%
subject 24 的准确率: 72.5%
subject 27 的准确率: 75.8%
subject 28 的准确率: 58.3%
subject 31 的准确率: 67.1%
subject 34 的准确率: 57.5%
subject 35 的准确率: 68.3%
所有被试的准确率： {'22': 0.6375, '24': 0.725, '27': 0.7583333333333333, '28': 0.5833333333333334, '31': 0.6708333333333333, '34': 0.575, '35': 0.6833333333333333}
所有被试的平均准确率: 66.2%
所有被试准确率（无名称）: 63.7,72.5,75.8,58.3,67.1,57.5,68.3


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_20_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 20
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 70.8%
subject 24 的准确率: 77.1%
subject 27 的准确率: 82.1%
subject 28 的准确率: 63.3%
subject 31 的准确率: 74.6%
subject 34 的准确率: 62.9%
subject 35 的准确率: 74.2%
所有被试的准确率： {'22': 0.7083333333333334, '24': 0.7708333333333334, '27': 0.8208333333333333, '28': 0.6333333333333333, '31': 0.7458333333333333, '34': 0.6291666666666667, '35': 0.7416666666666667}
所有被试的平均准确率: 72.1%
所有被试准确率（无名称）: 70.8,77.1,82.1,63.3,74.6,62.9,74.2


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_30_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 30
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 68.8%
subject 24 的准确率: 72.9%
subject 27 的准确率: 81.7%
subject 28 的准确率: 65.8%
subject 31 的准确率: 77.1%
subject 34 的准确率: 61.7%
subject 35 的准确率: 77.9%
所有被试的准确率： {'22': 0.6875, '24': 0.7291666666666666, '27': 0.8166666666666667, '28': 0.6583333333333333, '31': 0.7708333333333334, '34': 0.6166666666666667, '35': 0.7791666666666667}
所有被试的平均准确率: 72.3%
所有被试准确率（无名称）: 68.8,72.9,81.7,65.8,77.1,61.7,77.9


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_40_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 40
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 60.8%
subject 24 的准确率: 64.2%
subject 27 的准确率: 73.3%
subject 28 的准确率: 57.9%
subject 31 的准确率: 68.8%
subject 34 的准确率: 59.2%
subject 35 的准确率: 72.5%
所有被试的准确率： {'22': 0.6083333333333333, '24': 0.6416666666666667, '27': 0.7333333333333333, '28': 0.5791666666666667, '31': 0.6875, '34': 0.5916666666666667, '35': 0.725}
所有被试的平均准确率: 65.2%
所有被试准确率（无名称）: 60.8,64.2,73.3,57.9,68.8,59.2,72.5


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_50_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 50
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 61.7%
subject 24 的准确率: 63.7%
subject 27 的准确率: 73.8%
subject 28 的准确率: 57.5%
subject 31 的准确率: 67.5%
subject 34 的准确率: 58.3%
subject 35 的准确率: 70.8%
所有被试的准确率： {'22': 0.6166666666666667, '24': 0.6375, '27': 0.7375, '28': 0.575, '31': 0.675, '34': 0.5833333333333334, '35': 0.7083333333333334}
所有被试的平均准确率: 64.8%
所有被试准确率（无名称）: 61.7,63.7,73.8,57.5,67.5,58.3,70.8


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_60_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 60
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 62.1%
subject 24 的准确率: 65.4%
subject 27 的准确率: 71.7%
subject 28 的准确率: 59.2%
subject 31 的准确率: 68.8%
subject 34 的准确率: 59.6%
subject 35 的准确率: 70.4%
所有被试的准确率： {'22': 0.6208333333333333, '24': 0.6541666666666667, '27': 0.7166666666666667, '28': 0.5916666666666667, '31': 0.6875, '34': 0.5958333333333333, '35': 0.7041666666666667}
所有被试的平均准确率: 65.3%
所有被试准确率（无名称）: 62.1,65.4,71.7,59.2,68.8,59.6,70.4


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_70_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 70
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 56.7%
subject 24 的准确率: 62.1%
subject 27 的准确率: 67.1%
subject 28 的准确率: 54.6%
subject 31 的准确率: 63.7%
subject 34 的准确率: 57.5%
subject 35 的准确率: 65.8%
所有被试的准确率： {'22': 0.5666666666666667, '24': 0.6208333333333333, '27': 0.6708333333333333, '28': 0.5458333333333333, '31': 0.6375, '34': 0.575, '35': 0.6583333333333333}
所有被试的平均准确率: 61.1%
所有被试准确率（无名称）: 56.7,62.1,67.1,54.6,63.7,57.5,65.8


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_80_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 80
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 60.8%
subject 24 的准确率: 59.6%
subject 27 的准确率: 71.2%
subject 28 的准确率: 56.7%
subject 31 的准确率: 67.1%
subject 34 的准确率: 57.9%
subject 35 的准确率: 68.8%
所有被试的准确率： {'22': 0.6083333333333333, '24': 0.5958333333333333, '27': 0.7125, '28': 0.5666666666666667, '31': 0.6708333333333333, '34': 0.5791666666666667, '35': 0.6875}
所有被试的平均准确率: 63.2%
所有被试准确率（无名称）: 60.8,59.6,71.2,56.7,67.1,57.9,68.8


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_90_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 90
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 58.3%
subject 24 的准确率: 62.5%
subject 27 的准确率: 68.8%
subject 28 的准确率: 54.2%
subject 31 的准确率: 65.0%
subject 34 的准确率: 55.4%
subject 35 的准确率: 65.8%
所有被试的准确率： {'22': 0.5833333333333334, '24': 0.625, '27': 0.6875, '28': 0.5416666666666666, '31': 0.65, '34': 0.5541666666666667, '35': 0.6583333333333333}
所有被试的平均准确率: 61.4%
所有被试准确率（无名称）: 58.3,62.5,68.8,54.2,65.0,55.4,65.8


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_100_0.2.pth"
# 加载预训练模型
F1 = 20
D = 2
F2 = 40
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.2
alpha = 0
epsilon = 100
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 51.7%
subject 24 的准确率: 55.8%
subject 27 的准确率: 63.3%
subject 28 的准确率: 46.7%
subject 31 的准确率: 57.5%
subject 34 的准确率: 48.3%
subject 35 的准确率: 58.8%
所有被试的准确率： {'22': 0.5166666666666667, '24': 0.5583333333333333, '27': 0.6333333333333333, '28': 0.4666666666666667, '31': 0.575, '34': 0.48333333333333334, '35': 0.5875}
所有被试的平均准确率: 54.6%
所有被试准确率（无名称）: 51.7,55.8,63.3,46.7,57.5,48.3,58.8
